<img src="https://i.imgur.com/6U6q5jQ.png"/>

_____
<a id='home'></a>




# Introduction to Optimization for Decision Making


In [ ]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQHq0p2eTmxRWJjDmo1mUmdarYgIrEew4ieiVbIGQy-D_CyBw5rbbRUlRxwLKKaVQpRV9Hs8MGnz0X2/embed?start=false&loop=false&delayms=3000" frameborder="1" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

# Part 1: Maximization/Minimization 

Please, go to your _environment_ in Anacoda Navigator to install **glpk** and **pulp**  before runing the codes below.
Then, call the library:

In [ ]:
# pip show glpk pulp
# pip install glpk pulp

In [ ]:
import pulp as pp

1. **Initialize the MODEL**: just write the name and declare if it is maximization or minimization problem type.

In [ ]:
model = pp.LpProblem(name='refinery-problem', # just the name
                     sense=pp.LpMaximize) # type of problem

2. **Declare the VARIABLES**: The refinery model consists of these _variables_:

In [ ]:
# how much gas?
Gas = pp.LpVariable(name="Gas",  # just the name
                    lowBound=0,  # ensure non-negativity
                    cat='Continuous') # here: you accept decimal values

# how much oil?
Oil = pp.LpVariable(name="Oil",
                 lowBound=0,
                 cat='Continuous')

3. **Create function to OPTIMIZE**: The function is just the linear combination of the variables and their _given coefficients__: 

In [ ]:
GasCoeff=1.9
OilCoeff=1.5
obj_func = GasCoeff*Gas + OilCoeff*Oil

4. **Represent the constraints**: These are the rules the model (set of variables) must obey:

In [ ]:
# SUBJECT TO:
C1= pp.LpConstraint(name='Gas Constraint',   # just the name
                    e= 1*Gas - 2*Oil, rhs=0, # linear combination of constraint and rhs 
                    sense=pp.LpConstraintGE) # 'rule' >= 0 (LpConstraintGE)
C2= pp.LpConstraint(name='Oil Constraint',
                    e= 1*Oil, rhs=3000000,
                    sense=pp.LpConstraintGE) # 'rule' >= 3000000 (LpConstraintGE)
C3= pp.LpConstraint(name='Demand Constraint',
                    e= 1*Gas, rhs=6400000,
                    sense=pp.LpConstraintLE, )# 'rule' <= 6400000 (LpConstraintLE)

5. **Build MODEL**: Here you add (i) the objective function, and (ii) all the constraints:

In [ ]:
model += obj_func
model += C1
model += C2
model += C3


6. **Solve the MODEL**: Notice we are not using the _default solver_, we are explicitly usig **COIN_CMD**:

In [ ]:
solver_list = pp.listSolvers()
print(solver_list)

In [ ]:
solverToUse = pp.COIN_CMD(msg=False)
model.solve(solver=solverToUse);

You can create a summary like this:

In [ ]:
import pandas as pd

Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
Results

In [ ]:
#or
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

<div class="alert-success">

<strong>Exercise: The diet problem</strong> 

In [ ]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

# Part 2: Multicriteria Decision-Making

In [ ]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vR7GL_wF1eKRO0JgEUyIx5cxXUhTQ8ZM4F3AE1MLr7GYG33dwEobrLo6O2MaV2d7Cv47TaTgHghkhrV/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

In [2]:
# this in the folder
from ahp.ahp import AHP

In [3]:
import pandas as pd

pairwise_age=pd.read_excel("ahp_tables.xlsx",sheet_name='age', index_col=0)
pairwise_experience=pd.read_excel("ahp_tables.xlsx",sheet_name='experience', index_col=0)
pairwise_education=pd.read_excel("ahp_tables.xlsx",sheet_name='education', index_col=0)
pairwise_charisma=pd.read_excel("ahp_tables.xlsx",sheet_name='charisma', index_col=0)
pairwise_criteria=pd.read_excel("ahp_tables.xlsx",sheet_name='criteria', index_col=0)

In [4]:
pairwise_criteria

,Experience,Education,Charisma,Age
Experience,1.000000,4.000000,3.000000,7
Education,0.250000,1.000000,0.333333,3
Charisma,0.333333,3.000000,1.000000,5
Age,0.142857,0.333333,0.200000,1


In [5]:
# follow the same order as the criteria
data = [pairwise_experience.values.tolist(),
        pairwise_education.values.tolist(),
        pairwise_charisma.values.tolist(),
        pairwise_age.values.tolist(),
        pairwise_criteria.values.tolist()]

In [6]:
# instanciando o modelo
ahp = AHP()

# alimentando o modelo com os dados
ahp.fit(data)

In [7]:
Solution=pd.DataFrame(index=pairwise_charisma.index)
Solution['Score']=ahp.classificate()
Solution['Ranking']=Solution.Score.rank(ascending=False)
Solution

,Score,Ranking
Tom,0.359067,2.0
Dick,0.488307,1.0
Harry,0.152626,3.0
